In [1]:
import pandas as pd
import random
from rdflib.namespace import RDF, RDFS, FOAF, XSD
from rdflib import Graph
import pandas as pd
from rdflib import Namespace
from rdflib import Literal

In [2]:
authorsDF = pd.read_csv('data/authors.csv')
authorsPapersDF = pd.read_csv('data/authorsPapers.csv')
chairsDF = pd.read_csv('data/chairs.csv')
conferencesDF = pd.read_csv('data/conferences.csv')
editorsDF = pd.read_csv('data/editors.csv')
finalPapersDF = pd.read_csv('data/finalPapers.csv')
journalsDF = pd.read_csv('data/journals.csv')
papersDF = pd.read_csv('data/papers.csv')
reviewersDF = pd.read_csv('data/reviewers.csv')
reviewsDF = pd.read_csv('data/reviews.csv')
subjectsConferencesJournals = pd.read_csv('data/subjectsConferencesJournals.csv')
subjectDomainDF = pd.read_csv('data/subjectDomain.csv')
subjectsPapersDF = pd.read_csv('data/subjectsPapers.csv')
submittedPapersDF = pd.read_csv('data/submittedPapers.csv')

In [3]:
authorsPapersDF.head(2)

,authorUrl,paperId,authorId,authorName,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolume
0,6391996,p0,a0,E. Bolyen,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper,4
1,6307534,p0,a1,J. Rideout,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper,4


In [4]:
# Creating a graph
g = Graph()

In [5]:
# Create many URIRefs in the same namespace, i.e. URIs with the same prefix
LAB = Namespace("http://SDM_LAB3.org/")

In [6]:
# Bind the lab namespace to a prefix for more readable output
g.bind('lab',LAB)

#### Author -- [writes] --> Paper

In [7]:
authorsPapersDF.head(2)

,authorUrl,paperId,authorId,authorName,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolume
0,6391996,p0,a0,E. Bolyen,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper,4
1,6307534,p0,a1,J. Rideout,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper,4


In [8]:
# Adding author--[writes]--> paper instances to the graph
g.add((LAB.Author, LAB.writes, LAB.Paper))   
for k in range(len(authorsPapersDF['authorName'])):
    g.add((Literal(authorsPapersDF['authorName'][k]), LAB.writes, Literal(authorsPapersDF['paperTitle'][k])))

#### Paper -- [submitted] --> SubmittedPaper

In [9]:
submittedPapersDF.head(2)

,submissionId,paperId,paperTitle,paperAbstract,conferenceJournalId,conferenceJournalTitle,proceedingsVolume,decision
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,4,accepted
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,5,accepted


In [10]:
# Adding paper--[submitted]--> submittedPaper instances to the graph
g.add((LAB.Paper, LAB.submitted, LAB.SubmittedPaper))   
for k in range(len(submittedPapersDF['paperTitle'])):
    g.add((Literal(submittedPapersDF['paperTitle'][k]), LAB.writes, Literal(submittedPapersDF['paperTitle'][k])))

#### Paper -- [paperRelatedTo] --> SubjectDomain

In [11]:
subjectsPapersDF.head(2)

,subjectDomainName,paperId,subjectDomainId,paperUrl,conferenceJournalId,conferenceJournalTitle,paperTitle,paperAbstract,paperType,proceedingsVolume
0,Engineering,p0,d0,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper,4
1,Medicine,p0,d1,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper,4


In [12]:
# Adding paper--[paperRelatedTo]--> subjectDomain instances to the graph
g.add((LAB.Paper, LAB.paperRelatedTo, LAB.SubjectArea))
for k in range(len(subjectsPapersDF['paperTitle'])):
    g.add((Literal(subjectsPapersDF['paperTitle'][k]), LAB.paperRelatedTo, Literal(subjectsPapersDF['subjectDomainName'][k])))

#### Conference -- [isIn] --> Proceedings

In [13]:
conferencesDF.head(2)

,conferenceId,conferenceUrl,conferenceTitle,conferenceType,conferenceProceedings
0,c0,b55b50b1-aae7-47a7-b042-8aecc930073d,International Conference on Human Factors in C...,symposium,10
1,c1,2c9ecac6-f875-4a9b-acc2-10bd9f6782df,Technical Symposium on Computer Science Education,workshop,1


In [14]:
# Adding conference--[isIn]--> proceedings instances to the graph
g.add((LAB.Conference, LAB.isIn, LAB.Proceedings))
for k in range(len(conferencesDF['conferenceTitle'])):
    g.add((Literal(conferencesDF['conferenceTitle'][k]), LAB.isIn, Literal(conferencesDF['conferenceProceedings'][k])))

#### Journal -- [isOf] --> Volume

In [15]:
journalsDF.head(2)

,journalId,journalUrl,journalTitle,journalVolumes
0,j0,458166b3-de17-4bf3-bbbb-e53782de2f0f,Nature Biotechnology,4
1,j1,c6840156-ee10-4d78-8832-7f8909811576,IEEE Transactions on Knowledge and Data Engine...,0


In [16]:
# Adding journal--[isOf]--> volume instances to the graph
g.add((LAB.Journal, LAB.isOf, LAB.Volume))
for k in range(len(journalsDF['journalTitle'])):
    g.add((Literal(journalsDF['journalTitle'][k]), LAB.isOf, Literal(journalsDF['journalVolumes'][k])))

#### SubmittedPaper -- [isSubmittedTo] --> Venue

In [17]:
submittedPapersDF.head(2)

,submissionId,paperId,paperTitle,paperAbstract,conferenceJournalId,conferenceJournalTitle,proceedingsVolume,decision
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,4,accepted
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,5,accepted


In [18]:
# Adding submittedPaper--[isSubmittedTo]--> venue instances to the graph
g.add((LAB.SubmittedPaper, LAB.isSubmittedTo, LAB.Venue))
for k in range(len(submittedPapersDF['paperTitle'])):
    g.add((Literal(submittedPapersDF['paperTitle'][k]), LAB.isSubmittedTo, Literal(submittedPapersDF['conferenceJournalTitle'][k])))

#### SubmittedPaper -- [isAs] --> FinalPaper

In [19]:
finalPapersDF.head(2)

,submissionId,paperId,paperTitle,paperAbstract,conferenceJournalId,conferenceJournalTitle,proceedingsVolume,decision,finalPaperId
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,4,accepted,f0
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,5,accepted,f1


In [20]:
# Adding submittedPaper--[isAs]--> finalPaper instances to the graph
g.add((LAB.SubmittedPaper, LAB.isAs, LAB.FinalPaper))
for k in range(len(finalPapersDF['paperTitle'])):
    g.add((Literal(finalPapersDF['paperTitle'][k]), LAB.isAs, Literal(finalPapersDF['paperTitle'][k])))

#### FinalPaper -- [isPublishedInConference] --> Proceedings & FinalPaper -- [isPublishedInJournal] --> Volume

In [21]:
finalPapersDF.head(2)

,submissionId,paperId,paperTitle,paperAbstract,conferenceJournalId,conferenceJournalTitle,proceedingsVolume,decision,finalPaperId
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,4,accepted,f0
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,5,accepted,f1


In [22]:
conferencesDF.head(2)

,conferenceId,conferenceUrl,conferenceTitle,conferenceType,conferenceProceedings
0,c0,b55b50b1-aae7-47a7-b042-8aecc930073d,International Conference on Human Factors in C...,symposium,10
1,c1,2c9ecac6-f875-4a9b-acc2-10bd9f6782df,Technical Symposium on Computer Science Education,workshop,1


In [23]:
# Adding FinalPaper -- [isPublishedInConference] --> Proceedings & FinalPaper -- [isPublishedInJournal] --> Volume instances to the graph
g.add((LAB.FinalPaper, LAB.isPublishedInConference, LAB.Proceedings))
g.add((LAB.FinalPaper, LAB.isPublishedInJournal, LAB.Volume))
for k in range(len(finalPapersDF['paperTitle'])):
    if finalPapersDF['conferenceJournalId'][k][0] == 'c':
        g.add((Literal(finalPapersDF['paperTitle'][k]), LAB.isPublishedInConference, Literal(finalPapersDF['proceedingsVolume'][k])))
    else:
        g.add((Literal(finalPapersDF['paperTitle'][k]), LAB.isPublishedInJournal, Literal(finalPapersDF['proceedingsVolume'][k])))

#### Proceedings -- [proceedingsRelatedTo] --> SubjectDomain & Volumes -- [volumeRelatedTo] --> SubjectDomain

In [24]:
subjectsConferencesJournals.head(2)

,conferenceJournalTitle,conferenceJournalId,subjectDomainName
0,Nature Biotechnology,j0,"['Medicine', 'Biology', 'Engineering', 'Comput..."
1,Workshop on Learning from Authoritative Securi...,c52,"['Medicine', 'Sociology', 'Biology', 'Engineer..."


In [25]:
# Adding Proceedings -- [proceedingsRelatedTo] --> SubjectDomain & Volumes -- [volumeRelatedTo] --> SubjectDomain instances to the graph
g.add((LAB.Proceedings, LAB.proceedingsRelatedTo, LAB.SubjectDomain))
g.add((LAB.Volume, LAB.volumeRelatedTo, LAB.SubjectDomain))
for k in range(len(subjectsConferencesJournals['conferenceJournalTitle'])):
    curr_confJour_keyword = subjectsConferencesJournals['subjectDomainName'][k]
    if subjectsConferencesJournals['conferenceJournalId'][k][0] == 'c':
        for keyword in curr_confJour_keyword:
            g.add((Literal(subjectsConferencesJournals['conferenceJournalTitle'][k]), LAB.proceedingsRelatedTo, Literal(keyword)))
    else:
        for keyword in curr_confJour_keyword:
            g.add((Literal(subjectsConferencesJournals['conferenceJournalTitle'][k]), LAB.volumeRelatedTo, Literal(keyword)))

#### Creating Chair -- [handleConference] --> Conference & Editor -- [handleJournal] --> [Journal]

In [26]:
chairsDF.head(2)

,conferenceId,chairId,authorId,authorUrl,authorName,conferenceUrl,conferenceTitle,conferenceType,conferenceProceedings
0,c90,chair0,a5876,1753858,R. Casadio,768b87bb-8a18-4d9c-a161-4d483c776bcf,Computer Vision and Pattern Recognition,expertGroup,5
1,c90,chair15,a8803,3862899,A. Baylay,768b87bb-8a18-4d9c-a161-4d483c776bcf,Computer Vision and Pattern Recognition,expertGroup,5


In [27]:
# Adding Creating Chair -- [handleConference] --> Conference instances to the graph
g.add((LAB.Chair, LAB.handleConference, LAB.Conference))
for k in range(len(chairsDF['conferenceId'])):
    g.add((Literal(chairsDF['authorName'][k]), LAB.handleConference, Literal(chairsDF['conferenceTitle'][k])))

In [28]:
editorsDF.head(2)

,journalId,editorId,authorId,authorUrl,authorName,journalUrl,journalTitle,journalVolumes
0,j0,editor0,a9128,4309890,F. Lens,458166b3-de17-4bf3-bbbb-e53782de2f0f,Nature Biotechnology,4
1,j0,editor125,a2233,121828020,Ian M. Foster,458166b3-de17-4bf3-bbbb-e53782de2f0f,Nature Biotechnology,4


In [29]:
# Adding Editor -- [handleJournal] --> [Journal] instances to the graph
g.add((LAB.Editor, LAB.handleJournal, LAB.Journal))
for k in range(len(editorsDF['journalId'])):
    g.add((Literal(editorsDF['authorName'][k]), LAB.handleJournal, Literal(editorsDF['journalTitle'][k])))

#### Creating Handler -- [assigns] --> Reviewer: #TODO:

In [30]:
# Adding Creating Handler -- [assigns] --> Reviewer instances to the graph

#### Creating Reviewer -- [reviewed] --> ReviewText

In [31]:
reviewersDF.head(2)

,rId,submissionId,authorId,authorUrl,authorName,paperId,paperTitle,paperAbstract,conferenceJournalId,conferenceJournalTitle,proceedingsVolume,decision
0,r0,s0,a10203,2646507,H. Kälviäinen,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,4,accepted
1,r1,s0,a3119,1398779531,Régine Vignes-Lebbe,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,4,accepted


In [32]:
# Adding Reviewer -- [reviewed] --> ReviewText instances to the graph
g.add((LAB.Reviewer, LAB.reviewed, LAB.ReviewText))
for k in range(len(reviewersDF['rId'])):
    g.add((Literal(reviewersDF['authorName'][k]), LAB.reviewed, Literal(reviewersDF['conferenceJournalTitle'][k])))

#### Creating ReviewText -- [isReviewOf] --> SubmittedPaper

In [33]:
reviewsDF.head(2)

,reviewId,submittedPaperId,reviewText,reviewDecision,submittedPaperTitle
0,r0,s0,content of the reviewed text goes here....,accepted,"Reproducible, interactive, scalable and extens..."
1,r1,s1,content of the reviewed text goes here....,accepted,High-Dimensional Probability: An Introduction ...


In [34]:
# Adding ReviewText -- [isReviewOf] --> SubmittedPaper instances to the graph
g.add((LAB.ReviewText, LAB.isReviewOf, LAB.SubmittedPaper))
for k in range(len(reviewsDF['reviewId'])):
    g.add((Literal(reviewsDF['reviewText'][k]), LAB.isReviewOf, Literal(reviewsDF['submittedPaperTitle'][k])))

#### Additional triples - TODO:

#### Paper -- [paperTitle] --> String 
g.add((LAB.Paper, LAB.paperTitle, XSD.String))

#### Paper -- [paperAbstract] --> String
g.add((LAB.Paper, LAB.paperAbstract, XSD.String))

#### Author -- [authorName] --> String 
g.add((LAB.Author, LAB.authorName, XSD.String))

#### Journal -- [journalTitle] --> String 
g.add((LAB.Journal, LAB.journalTitle, XSD.String))

#### Conference -- [conferenceTitle] --> String 
g.add((LAB.Conference, LAB.conferenceTitle, XSD.String))

#### Proceedings -- [proceedingName] --> String 
g.add((LAB.Proceedings, LAB.proceedingName, XSD.String))

#### Proceedings -- [proceedingYear] --> int 
g.add((LAB.Proceedings, LAB.proceedingYear, XSD.int))

#### Volume -- [volumeName] --> String 
g.add((LAB.Proceedings, LAB.proceedingName, XSD.String))

#### Volume -- [volumeYear] --> int 
g.add((LAB.Volume, LAB.volumeYear, XSD.int))

#### SubjectDomain -- [keywords] --> String 
g.add((LAB.SubjectDomain, LAB.keywords, XSD.String))

#### ReviewText -- [comment] --> String 
g.add((LAB.ReviewText, LAB.comment, XSD.String))

#### ReviewText -- [decision] --> boolean 
g.add((LAB.ReviewText, LAB.decision, XSD.boolean))

#### FinalPaper -- [paperAcceptanceDate] --> date
g.add((LAB.FinalPaper, LAB.paperAcceptanceDate, XSD.date))

In [36]:
# g.serialize(format = 'ttl')